In [9]:
import requests
import pandas as pd
import plotly.express as px
import time
import os


In [2]:

def Get_data():
    Station_informations_website = "https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/station_information.json"
    Real_time_informations_website = "https://velib-metropole-opendata.smovengo.cloud/opendata/Velib_Metropole/station_status.json"

    Stations_informations_file = requests.get(Station_informations_website).json() # Get the informations about the stations in Paris
    Real_time_informations_file = requests.get(Real_time_informations_website).json() # Get the real time informations

    # Extract the list of stations
    stations = Stations_informations_file['data']['stations']

    # Create a DataFrame from the extracted data
    df_stations = pd.DataFrame(stations)

    # Select only the relevant columns and rename them appropriately
    df_stations = df_stations[['capacity', 'name', 'lon', 'lat', 'station_id']]
    df_stations.columns = ['Capacity', 'Name', 'Longitude', 'Latitude', 'Station_ID']

    # Extract the list of stations
    stations2 = Real_time_informations_file['data']['stations']

    # Create a DataFrame from the extracted data
    df_real_time = pd.DataFrame(stations2)

    # Select only the relevant columns and rename them appropriately
    df_real_time = df_real_time[['numBikesAvailable','num_bikes_available_types' ,'num_docks_available', 'station_id','is_installed', 'is_returning', 'is_renting']]
    df_real_time.columns = ['bikes_available','num_bikes_available_types' ,'Docks_available', 'Station_ID','is_installed', 'is_returning', 'is_renting']

    df_real_time["Mechanical"] = 0
    df_real_time["Electrical"] = 0




    for x in range(0, len(df_real_time)):
        df_real_time["Mechanical"][x] = df_real_time["num_bikes_available_types"][x][0]["mechanical"]
        df_real_time["Electrical"][x] = df_real_time["num_bikes_available_types"][x][1]["ebike"]
        
    df_real_time = df_real_time.drop(columns=['num_bikes_available_types'])
    
    # Merge the two dataset
    df_final = pd.merge(df_stations, df_real_time, on='Station_ID', how='inner')

    # Adding the ration between electrical and manual
    df_final["Ebike_ratio"] = df_final["Electrical"] / df_final["bikes_available"]
    df_final["Ebike_ratio"] = df_final["Ebike_ratio"].fillna(0.5)
    return df_final


In [3]:
Data = Get_data()

C:\Users\jerem\AppData\Local\Temp\ipykernel_18744\772008188.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_real_time["Mechanical"][x] = df_real_time["num_bikes_available_types"][x][0]["mechanical"]
C:\Users\jerem\AppData\Local\Temp\ipykernel_18744\772008188.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_real_time["Electrical"][x] = df_real_time["num_bikes_available_types"][x][1]["ebike"]
C:\Users\jerem\AppData\Local\Temp\ipykernel_18744\772008188.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydat

In [12]:
# Votre échelle de couleurs personnalisée et la création de la carte
color_scale = [(0, "green"), (1, "blue")]

fig = px.scatter_mapbox(Data, 
                        lat="Latitude", 
                        lon="Longitude", 
                        hover_name="Name", 
                        hover_data=["Name","bikes_available","Mechanical","Electrical", "Capacity", "Station_ID", "Ebike_ratio"],
                        color="Ebike_ration",
                        color_continuous_scale=color_scale,
                        zoom=10, 
                        height=800,)

# Set marker size
fig.update_traces(marker=dict(size=15))  # Fixed size for all markers

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.update_traces(showlegend=False)

fig.show()

time.sleep(5)
